## Построение baseline-решений

Вам предстоит построить модель, прогнозирующую клиентов, склонных к оттоку. В качестве ответа, вам нужно будет не просто выбрать к какому классу ('1' - отток, '-1' - не отток) относится пользователь, но и дать вероятностную оценку, т.е. оценить с какой вероятностью клиент относится к классу отток.

Обратите внимание на данные, с которыми вы столкнетесь: они содержат пропущенные значения и категориальные переменные.



Оценка качества проводится с помощью метрики ROC-AUC. 

В этом задании вам предстоит поучаствовать в соревновании на kaggle inclass

Перейдите по ссылке на страницу соревнования:

https://inclass.kaggle.com/c/telecom-clients-prediction2

И приступайте!

В соревновании вы будете работать с той же выборкой, что и ранее, поэтому воспользуйтесь результатами, полученными на предыдущих неделях. Для успешного участия в соревновании необходимо преодолеть по качеству beseline решение.

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, model_selection, metrics, ensemble
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.utils import shuffle
import xgboost as xgb

In [334]:
train = pd.read_csv('Train.csv')
train

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,label
0,NaN,NaN,NaN,NaN,NaN,805.0,7.0,NaN,NaN,NaN,...,fXLavGi,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
1,NaN,NaN,NaN,NaN,NaN,854.0,7.0,NaN,NaN,NaN,...,UulNbBP,LM8l689qOp,NaN,NaN,Qu4f,02N6s8f,R4y5gQQWY8OodqDV,am7c,NaN,-1
2,NaN,NaN,NaN,NaN,NaN,560.0,21.0,NaN,NaN,NaN,...,s4wQZmh,LM8l689qOp,NaN,ELof,Qcbd,RAYp,F2FyR07IdsN7I,am7c,NaN,-1
3,NaN,NaN,NaN,NaN,NaN,14.0,0.0,NaN,NaN,NaN,...,6iNOEra,LM8l689qOp,NaN,NaN,Qu4f,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
4,NaN,NaN,NaN,NaN,NaN,1806.0,7.0,NaN,NaN,NaN,...,uyDq6g3,LM8l689qOp,NaN,ELof,TNEC,02N6s8f,ib5G6X1eUxUn6,am7c,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,NaN,NaN,NaN,NaN,NaN,721.0,7.0,NaN,NaN,NaN,...,i9SE4Qn,LM8l689qOp,NaN,xG3x,Aoh3,RAYp,F2FyR07IdsN7I,am7c,NaN,-1
29996,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,LiUPLfR,LM8l689qOp,NaN,NaN,szEZ,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
29997,NaN,NaN,NaN,NaN,NaN,1897.0,7.0,NaN,NaN,NaN,...,k0gYyp5,LM8l689qOp,NaN,kG3k,Qu4f,RAYp,F2FyR07IdsN7I,mj86,NaN,1
29998,NaN,NaN,NaN,NaN,NaN,700.0,0.0,NaN,NaN,NaN,...,8C02pc5,LM8l689qOp,NaN,NaN,Xa3G,RAYp,F2FyR07IdsN7I,NaN,NaN,-1


In [3]:
num_columns = train.columns[:190]
cat_columns = train.columns[190:230]
num_columns

Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8', 'Var9',
       'Var10',
       ...
       'Var181', 'Var182', 'Var183', 'Var184', 'Var185', 'Var186', 'Var187',
       'Var188', 'Var189', 'Var190'],
      dtype='object', length=190)

### Отбор числовых признаков

In [111]:
def corrwithbin(df, feature, temp):
    value = temp.iloc[:,0]
    label = temp.iloc[:,1]
    # будем отбирать только те признаки, в которых есть не менее 10000 not Nan величин
    if len(value) < 5000:
        corr =0
    else:
        corr = abs(value[label == 1].mean()) - abs(value[label == -1].mean())
        df.loc[len(df)] = [feature, np.abs(corr), len(value[label == 1]), len(value[label == -1])]
    return df

In [112]:
correlation = pd.DataFrame(columns= ['feature', 'corr', 'label = 1', 'label = -1'])
for i in num_columns:
    temp = train[[i,'label']].dropna()
    correlation = corrwithbin(correlation, i, temp)

In [203]:
correlation.dropna(inplace=True)
top20_corr = correlation.sort_values(['corr'], ascending = False)[:60]
top_feature = top20_corr.feature.to_list()

In [344]:
def num_features(df, top_features):
    for col in num_columns:
        if col not in top_feature:
            df = df.drop(col,axis=1)
        else:
           # df = df.fillna(0)
            mean = df[col].dropna().mean()
            df[col] = df[col].fillna(mean)
            scaler = StandardScaler()
            df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))
    return df

In [377]:
data = train.copy()
data = num_features(data, top_feature)

In [322]:
data.shape

(30000, 83)

### Отбор категориальных признаков

In [45]:
def label_encoder(df):
    labelencoder = LabelEncoder()
    for i in good_cat_feature:
        if i in df.columns:
            df[i] = labelencoder.fit_transform(df[i].astype(str))
       
    return df

In [13]:
def undersampling(df):
    data_churn = df[df['label'] == 1]
    data_nochurn = df[df['label'] == -1]
    countFirst = len(data_churn)
    data_churn = data_churn.append(data_nochurn.iloc[:countFirst,:])
    data_churn = shuffle(data_churn)
    return data_churn

In [342]:
def one_hot_encoder(df):
    encoder = OneHotEncoder()
    for i in good_cat_feature:
        if i in df.columns:
            var = df[[i]]
            new_var = encoder.fit_transform(var).toarray()
            df = pd.concat(
                [
                    df, 
                    pd.DataFrame(
                        new_var,
                        columns = [i+'-'+str(j) for j in range(len(encoder.categories_[0]))]
                    )
                ], 
                axis=1,
                sort=False,
            )
            df = df.drop(i, axis=1)
    return df

In [372]:
good_cat_feature = ['Var205', 'Var210', 'Var211', 'Var218', 'Var223', 'Var225']

In [76]:
def cat_features(df):
    for col in cat_columns:
        if col in good_cat_feature:
            df = df.fillna('0')
        else:
            df = df.drop(col,axis=1)
    return df

In [378]:
# отбираем категориальные признаки
data = cat_features(data)
# кодируем категориальные признаки

#data1=  label_encoder(data)

data1 = one_hot_encoder(data)
data1.drop('label', axis=1, inplace=True)
data1['label'] = data.label

# # уравновешиваем классы 1 и -1
prep_data = undersampling(data1)

In [379]:
# разделяем данные
x = prep_data.iloc[:,:-1]
y = prep_data.iloc[:,-1:]

In [380]:
prep_data

,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,...,Var223-0,Var223-1,Var223-2,Var223-3,Var223-4,Var225-0,Var225-1,Var225-2,Var225-3,label
87,-0.196387,-1.145585e+00,-0.485185,-0.076427,-0.070982,0.170864,0.300529,-2.589834e-01,-0.256734,-9.017241e-01,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1
1361,-0.586527,-1.145585e+00,-0.485185,-0.214427,-0.208822,-0.508048,-0.428562,-1.972710e+00,-0.256734,9.295296e-01,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
234,-0.113730,2.424859e+00,1.759694,-0.214427,-0.208822,0.170864,0.128978,-4.455398e-02,-0.256734,-8.364566e-01,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1
28705,-0.183161,4.456295e-02,-0.403667,-0.223052,-0.217437,-0.508048,-0.471450,6.833100e-01,-0.256734,2.191583e-01,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1
14427,0.000000,-1.510091e-16,0.000000,0.000000,0.000000,0.000000,0.000000,-1.893893e-13,0.000000,1.632691e-16,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10943,0.372293,1.234711e+00,-0.306473,0.208197,0.213312,-0.055440,-0.171236,8.615399e-02,1.536277,-1.317723e-02,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
24075,0.405356,-1.145585e+00,-0.483617,0.182322,0.187467,0.170864,0.686519,-6.160810e-01,-0.256734,-8.780700e-01,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1
22315,-0.616283,-1.145585e+00,-0.485185,-0.481802,-0.475886,-0.508048,-0.514338,1.176241e+00,-0.256734,-9.025572e-01,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
1894,0.000000,-1.510091e-16,0.000000,0.000000,0.000000,0.000000,0.000000,-1.893893e-13,0.000000,1.632691e-16,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1


### Строим решение
Для решения применяется xgboost и gradientboosting. Обработка тестовых данных выполняется алгоритмом с большим значением метрики ROC_AUC на обучающей выборке после определения оптимальных значений гиперпараметров при помощи gridsearch

In [18]:
skf = model_selection.StratifiedKFold(n_splits=5)

In [19]:
# рассчет метрик по кросс-валидации
def get_model(clf, skf, x, y):
    roc_auc = []
    f1_score = []
    precision = []
    recall = []
    for train, test in skf.split(x,y):
        clf.fit(x.iloc[train,:], y[train])
        y_pred = clf.predict(x.iloc[test,:])
        y_pred_prob = clf.predict_proba(x.iloc[test,:])[:,1]
        precision.append(metrics.precision_score(y[test], y_pred))
        recall.append(metrics.recall_score(y[test], y_pred))
        f1_score.append(metrics.f1_score(y[test], y_pred))
        roc_auc.append(metrics.roc_auc_score(y[test], y_pred_prob))
        
    print('f1_score:', np.mean(f1_score))
    print('precision:', np.mean(precision))
    print('recall:', np.mean(recall))
    print('roc_auc:', np.mean(roc_auc))
        
    return roc_auc

In [143]:
# параметры для GridSearch для xgboost
grad_xgb_params = {
    'n_estimators' : np.arange(100, 1000, 200),
    'learning_rate' : [0.1, 0.01, 0.05],
    'subsample' : [0.3, 0.5, 0.8, 1.0],
    'max_depth' : [3, 5, 8]
}

In [381]:
# xgb baseline
clf_best= xgb.XGBClassifier(random_state=12)
roc_auc = get_model(clf_best, skf, x, y.label.values)

f1_score: 0.635943876628637
precision: 0.6278500737280832
recall: 0.6445604395604395
roc_auc: 0.6812033888609944


In [144]:
# поиск по сетке
grid_cv_xgb = model_selection.GridSearchCV(gbm, grad_xgb_params, n_jobs=-1, verbose=10, cv=skf, scoring='roc_auc')
grid_cv_xgb.fit(x, y.label.values)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.4s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   44.2s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   58.3s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  4

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster=None,
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=0.300000012,
                                     max_delta_step=0, max_depth=6,
                                     min_child_weight=1, miss...
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, subsample=1,
                                     tree_method=None,
                                     validate_parameters=False,
                                     verbosity=None),
             iid='dep

In [382]:
print('best score:', grid_cv_xgb.best_score_)
print('best param:', grid_cv_xgb.best_params_)

best score: 0.7349279782295977
best param: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 700, 'subsample': 0.8}


In [383]:
# gradientboosting baseline
clf = ensemble.GradientBoostingClassifier(random_state=12)
roc_auc = get_model(clf, skf, x, y.label.values)

f1_score: 0.6747154680716168
precision: 0.6639771595203346
recall: 0.6858424908424908
roc_auc: 0.7295449697786308


In [139]:
# параметры для GridSearch для gradientboosting
grad_gb_params = {
    'n_estimators' : np.arange(100, 1000, 200),
    'learning_rate' : [0.1, 0.01, 0.05],
    'subsample' : [0.3, 0.5, 0.8, 1.0],
    'max_depth' : [3, 5, 8]
}

In [140]:
# поиск по сетке
grid_cv_gb = model_selection.GridSearchCV(clf, grad_gb_params, n_jobs=-1, verbose=10, cv=skf, scoring='roc_auc')
grid_cv_gb.fit(x, y.label.values)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   49.8s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  5

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2...
                                                  presort='deprecated',
                                                  random_state=12,
                

In [384]:
print('best score:', grid_cv_gb.best_score_)
print('best param:', grid_cv_gb.best_params_)

best score: 0.7327179699036263
best param: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 700, 'subsample': 0.8}


###  Тестовые данные

In [385]:
test_data_kaggle = pd.read_csv('orange_small_churn_test_data.csv',)
test_data_kaggle.drop('ID',axis=1, inplace=True)

In [386]:
# преобразование тестовых данных
test_data = num_features(test_data_kaggle, top_feature)
test_data = cat_features(test_data)
test_data1 = one_hot_encoder(test_data)

In [389]:
# построение eslimator'a
clf_best= xgb.XGBClassifier(random_state=12, learning_rate=0.01, max_depth=3, n_estimators=700, subsample=0.8)
clf_best.fit(x, y.label.values)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=700, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=12, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=0.8, tree_method=None,
              validate_parameters=False, verbosity=None)

In [391]:
# получение вероятностой оценки для классов
y_test = pd.DataFrame({'ID': test_data1.index, 'result': clf_best.predict_proba(test_data1)[:,1]})

In [315]:
y_test

,ID,result
0,0,0.453659
1,1,0.516800
2,2,0.252373
3,3,0.398921
4,4,0.194796
...,...,...
9995,9995,0.259260
9996,9996,0.381449
9997,9997,0.327006
9998,9998,0.239219


In [394]:
y_test.to_csv('answer.csv', index=False)